# Running 200MHz Receptor Correlation
###### Last Updated: 22/04/24

This notebook will run through running through a 200Mhz receptor correlation, using 4 Talon boards.

## 1 Prerequisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry](https://python-poetry.org/docs/basic-usage/#installing-dependencies), and after that we can grab the imports required.

In [ ]:
import json
import os
from time import sleep

from tango import Database, DevFailed, DeviceProxy

As we will need to wait for a number of stages to be reached by the devices, we use this to wait for device state

In [ ]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

### 1.1 Running on PSI


To run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [ ]:
!kubectl get ns | grep ska-mid-psi

In [ ]:
!kubectl -n $ns get pods

### 1.2 Loading in Variables/Checking Taranta

Now, we set the variables we'll need for the run. First the constants that should not need to be changed unless debugging:

In [ ]:
# --Required Vars--
# Non-SPD namespace
ns = ""
# The boards we will be using for the test. Should only use either 1,2,3,4 or 5,6,7,8 to prevent issues with assignment.
target_boards = []
# Set to 0 for off, 1 for on
simulation_mode = 0

# Slim receptor config, leave blank if using default (1 board)
slim_fs_config = "fs_slim_4vcc_1fsp.yaml"
#slim_fs_config = ""
slim_vis_config = ""

# Delay Model file, set to alt if using one board
delay_model_filename = "delay_model_4r.json"
#delay_model_filename = "delay_model.json"

In [ ]:
# MAC address for the boards.
TARGET_MAC_ADDRESS = "08:c0:eb:9d:47:78"
# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG_FOLDER = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG_FOLDER = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
SLIM_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "slim_config")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]
BOARDS_ALT_FORMAT = ",".join(str(x) for x in target_boards)

With these set, we can now check the taranta dash to monitor the boards as we run through the rest of the notebook.

In [ ]:
url = "https://142.73.34.170/" + ns + "/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print(
    "You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=660de6afb20f1600120ec597&mode=run".format(
        ns
    )
)

At this point, all components should be in the `disabled` state.

## 2 Initial Setup


First we want to set up the target boards that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `target_boards` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [ ]:
TANGO_HOST = "databaseds-tango-base." + ns + ".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

We then load in the locations of local JSON files for configuration:

In [ ]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG_FOLDER, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG_FOLDER, "assign_resources.json")
SCAN_CONFIG_FILE = os.path.join(TMC_CONFIG_FOLDER, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG_FOLDER, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG_FOLDER, delay_model_filename)
SLIM_CONFIG_FILE = os.path.join(SLIM_CONFIG_FOLDER, slim_fs_config)

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    SCAN_CONFIG_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
    SLIM_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [ ]:
if slim_fs_config != "":
    print("Loading custom SLIM fs config")
    !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml
else:
    print("SLIM fs will use defaults for this test.")
if slim_vis_config != "":
    print("Loading custom SLIM vis config")
    !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml
else:
    print("SLIM vis will use default config")

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [ ]:
# Setup the device proxies targeting bite and deployer
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

Next we load in the hardware configuration depending on the talon boards selected. If the higher number boards are used, we need to use the swapped config file, and then modify each board value to match the swap file.

**DO NOT mix higher number boards with lower number ones (2,3,4,5), as this will cause issues with the hardware config**

In [ ]:
if any(i > 4 for i in target_boards):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards = list(map(lambda x: x - 4, target_boards))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")
BOARDS_ALT_FORMAT = ",".join(str(x) for x in target_boards)

Now we copy this file into the controller pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

## 3 Deploying

### 3.1 Deploying Using the Command Line

To use the old method for running the deployer commands, we use kubectl's exec function to run commands on the relevant pod, using the deployer script and passing the relevant commands to it as arguments.

In [ ]:
print("Targeting boards: {}".format(BOARDS_ALT_FORMAT))
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=$BOARDS_ALT_FORMAT
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --download-artifacts
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --config-db

Ensure that the download steps runs, and reaches completion.

### 3.2 Deploying Using the Deployer Device

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [ ]:
deployer_tango.targetTalons = target_boards
print(deployer_tango.targetTalons)

With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [ ]:
deployer_tango.generate_config_jsons()

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [ ]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)

To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [ ]:
!kubectl logs -n $ns ds-deployer-deployer-0 | grep 'Finished downloading'

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [ ]:
deployer_tango.configure_db()

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## 4 Uploading Controller Settings and Starting the Boards

Now that we have the requried devices deployed and exported, we can set up the DeviceProxies for the devices we'll use in the next steps:

In [ ]:
control_tango = DeviceProxy("mid-csp/control/0")
print(control_tango.status())
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.status())
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())

**Before running this step and the following steps, ensure that you have checked out the boards you have selected to use!**
***

We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [ ]:
# Set relevant values on the mid-csp controller
control_tango.adminMode = 0
sleep(1)
control_tango.write_attribute("cbfSimulationMode", simulation_mode)
control_tango.cbfSimulationMode = simulation_mode
sleep(1)

if (
    control_tango.read_attribute("adminMode").value == 0
    and control_tango.read_attribute("cbfSimulationMode").value == 0
):
    print("Set on, sim mode off!")
elif (
    control_tango.read_attribute("adminMode").value == 0
    and control_tango.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Checking the status dashboard, it should now display that all devices are both OFF and that the simulationstate is FALSE.

 Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command: 

In [ ]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print(json.dumps(data))

In [ ]:
upload_result = CBF_tango.InitSysParam(json.dumps(data))
print(upload_result[1])
print(CBF_tango.sysParam)

Now, we turn ON the Controller by passing it the device we want to turn on, and letting it run for 45ms to give the boards time to power on.

In [ ]:
print(control_tango.status())
control_tango.set_timeout_millis(100000)
target = ["mid_csp_cbf/sub_elt/controller"]
control_tango.On(target)
if simulation_mode == 1:
    sleep(5)
else:
    sleep(55)
print(control_tango.status())

In [ ]:
print(control_tango.status())

After running this step, check with the Taranta dashboard to check that the boards are started.

## 5 BITE Commands

### 5.1 Running Commands in BITE through Command Line Arguments (Old Method)

This is the older method for running BITE commands, utilizing kubectl to run python files directly:

First, we copy in required files for multi-board LSTV generation:

In [ ]:
CBF_TARGET_DIRECTORY = "/app/images/ska-mid-cbf-engineering-console-bite/test_parameters"
CBF_BASE_DIR = "/app/images/ska-mid-cbf-engineering-console-bite"
TEST_DATA_DIR = "test_parameters"
print(os.path.isdir(TEST_DATA_DIR))

In [ ]:
!kubectl exec ec-bite -n $ns -- rm -rf $CBF_TARGET_DIRECTORY
!kubectl cp $TEST_DATA_DIR $ns/ec-bite:$CBF_BASE_DIR

First we configure, specifying the MAC address of the boards and the boards to configure.

In [ ]:
!kubectl exec -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$BOARDS_ALT_FORMAT --bite_mac_address=$TARGET_MAC_ADDRESS --input_data=$BOARDS_ALT_FORMAT

### 5.2 Running Commands through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [ ]:
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

Now that we have confirmed it is running, we load in what board we want to configure.

In [ ]:
bite_tango.boards = target_boards
bite_tango.bite_mac_address = TARGET_MAC_ADDRESS

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
# print(bite_tango.command_inout("generate_bite_data"))

# For now run the backup
!kubectl exec -ti -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$BOARDS_ALT_FORMAT --bite_mac_address=$TARGET_MAC_ADDRESS

In [ ]:
!kubectl -n $ns logs ds-bite-bite-0 | grep generate_bite_data

With the bite data generated, we can then start generating LSTV playback data:

In [ ]:
bite_tango.start_lstv_replay()

We can then check the logs to ensure the LSTV geneartion runs correctly.

In [ ]:
!kubectl -n $ns logs ds-bite-bite-0 | grep start_lstv_replay

## 6 Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

For the loaded data, we must ensure the right boards are selected, modifying the data for the receptor ID based on the Talon board selected.

In [ ]:
# Load in the data from the file.
subarray_tango = DeviceProxy("mid-csp/subarray/01")
with open(ASSIGN_CSP_RESOURCES_FILE, encoding="utf-8") as init_file:
    config_dict = json.load(init_file)

# In order to use the correct receptor, we modify the assign_resources data to use the correct receptor based on the board we're using
print(json.dumps(config_dict))
config_dict["dish"]["receptor_ids"] = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards))
print(json.dumps(config_dict))

Next, we run the actual command to pass in the resources:

In [ ]:
subarray_tango.AssignResources(json.dumps(config_dict))
wait_for_state(subarray_tango, 2)

## 7 SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

First, we set up a proxy to the SDP device:

In [ ]:
sdp_tango = DeviceProxy("test-sdp/subarray/01")
sdp_tango.On()
print(sdp_tango.Status())

Now, we assign resources as we did with the subarray, as with the csp, we map the receptor boards based on our checked out board.

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    resources_json = json.load(f)
resources_json["dish"]["receptor_ids"] = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards))
resources_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards)
)

In [ ]:
sdp_only_json = resources_json["sdp"]
dish_only_json = resources_json["dish"]
print(json.dumps(sdp_only_json, indent=1))
print("================================")
print(json.dumps(dish_only_json, indent=1))

And run the command to assign resources:

In [ ]:
sdp_tango.AssignResources(json.dumps(sdp_only_json))

Next we send the configure command along with the required config file, making sure we are out of the 'RESOURCING' state.

In [ ]:
wait_for_state(sdp_tango, 2)

In [ ]:
with open(SCAN_CONFIG_FILE, encoding="utf-8") as f:
    sdp_configuration = json.load(f)
    sdp_configuration = sdp_configuration["sdp"]
print(json.dumps(sdp_configuration))

In [ ]:
sdp_tango.Configure(json.dumps(sdp_configuration))

This step may FAULT for a bit.

In [ ]:
wait_for_state(sdp_tango, 4)

We then start the SDP output via the Scan command:

In [ ]:
with open(SCAN_FILE, encoding="utf-8") as f:
    sdp_scan = json.load(f)
    sdp_scan = sdp_scan["sdp"]
print(json.dumps(sdp_scan))

In [ ]:
sdp_tango.Scan(json.dumps(sdp_scan))
print(sdp_tango.obsState)

After running this step, a pod for receiving visibilities will be launched, grab the IP of this pod and place it in the output_host var in the configure_scan json file.

In [ ]:
# Grab the pod name
!kubectl -n $ns-sdp get pods | grep vis-receive

In [ ]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00059-vis-receive-00-0"
!kubectl -n $ns-sdp describe pod $vis_pod | grep net1 -A 3

Grab the IP from this, and write it to a var we'll use later to configure the CSP.

In [ ]:
output_host = "10.50.1.34"

With this pod, we will also want to prep for monitoring the pod for when it recivies the visibilities

Run this command, then use the output in a separate terminal to enter the correct pod:

In [ ]:
!echo kubectl exec -n $ns-sdp -ti $vis_pod -- bash

We can then run the following commands in this pod to start monitoring for the correct traffic:

## 8 Starting up the CSP Scan

Now on the CSP side we can load in the corresponding Scan and cofig files, modifying the config to prep for our test.

In [ ]:
with open(SCAN_CONFIG_FILE, encoding="utf-8") as f:
    csp_configure_scan = json.load(f)["csp"]
with open(SCAN_FILE, encoding="utf-8") as f:
    csp_scan = json.load(f)["csp"]
# csp_configure_scan["common"]["config_id"] = "4 receptor, band 1, 1 FSP, no options"
csp_configure_scan["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
# csp_configure_scan["subarray"]["subarray_name"] = "4 receptors"
csp_configure_scan["subarray"]["subarray_name"] = "1 receptor"
# Write FSP and related data
csp_configure_scan["cbf"]["fsp"][0]["fsp_id"] = 1
csp_configure_scan["cbf"]["fsp"][0]["zoom_factor"] = 1
csp_configure_scan["cbf"]["fsp"][0]["zoom_window_tuning"] = 450000
csp_configure_scan["cbf"]["fsp"][0]["channel_offset"] = 0
csp_configure_scan["cbf"]["fsp"][0]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards)
)
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][0] = 0
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][1] = output_host

csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][0] = 0
csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][1] = 21000
csp_configure_scan["cbf"]["fsp"][0]["output_port"][0][2] = 1

In [ ]:
print(json.dumps(csp_configure_scan, indent=1))
print("==============")
print(json.dumps(csp_scan, indent=1))

In [ ]:
print("Observation state: {}".format(subarray_tango.obsState))
subarray_tango.Configure(json.dumps(csp_configure_scan))
sleep(5)
print("Observation state: {}".format(subarray_tango.obsState))

Then we start LSTV replay, check the logs and get the epoch value to use later (`INFO: start_utc_time_offset = start_utc_time.unix_tai - ska_epoch_tai = <EPOCH VALUE TO COPY>`)

In [ ]:
!kubectl exec -ti -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=$BOARDS_ALT_FORMAT --input_data=$BOARDS_ALT_FORMAT

We then store this epoch value to configure the delay model.

In [ ]:
target_epoch = 768090142.0

We then load this into the delay model, and change the `epoch` value to the one generated by the BITE LSTV replay start command, and the `receptor` to match our LSTV gen and receptor ID:

In [ ]:
with open(CSP_DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
print(json.dumps(delay_model, indent=1))
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model, indent=1))

In [ ]:
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delayModelProxy.write_attribute("delayModel", json.dumps(delay_model))

And then the matching command in the subarray:

In [ ]:
subarray_tango.Scan(json.dumps(csp_scan))

In [ ]:
print("Observation state: {}".format(subarray_tango.obsState.name))

Checking the dashboard, the dish should now be in the SCANNING status.

## 9 Checking Visibilities

With the devices scanning and linked, we can monitor the output via monitoring the network packets that come from it. Use the tcpdump running terminal to check that packet lengths are correct:
- UDP, length 136.
- UDP, length 7040.

Also check that the visibility pod is writing data.

## 10 Cleanup

Once we're satisfied with the results, we can stop the scans and shut down the boards.

In [ ]:
sdp_tango.EndScan()

In [ ]:
sdp_tango.End()

In [ ]:
subarray_tango.EndScan()
sleep(5)

In [ ]:
subarray_tango.End()

In [ ]:
subarray_tango.GoToIdle()
subarray_tango.ReleaseAllResources()

In [ ]:
control_tango.Off(target)

If the boards are on, use the LRU command to turn them off.

Now that we're done, free up dev resources on PSI by deleting your ns.

In [ ]:
!kubectl delete ns $ns
!kubectl delete ns $ns-sdp

In [ ]:
!kubectl get ns | grep ska-mid-psi

🎉 Congrats, you've now run the Auto Correlator demo!